# Анализ новостей

Это домашнее задание будет состоять из трёх этапов. Чисто формально оно состоит из трёх заданий. Каждое оценивается по 10-бальной шкале. Задания можно сдавать по-отдельности. 

Работать мы будем [с датасетом новостей](https://yadi.sk/d/uuyDA2hOdC9VHA) с РИА-новости за 2018 год. Он состоит из двух частей: 

* Новости с сайта https://ria.ru, заботливо собранные мной для вас.
* Новости из оффициальной группы РИА-новости вконтакте, а также комментарии к ним. Они тоже собирались для вас мной с заботой. 

__Внимание!__ Попытайтесь подгружать в тетрадку только те данные, которые вам необходимы на данном этапе работы. Также рекомендуется сохранять промежуточные результаты, чтобы впоcледствии не терять время попусту из-за умершего ядра. 

# 1. Знакомство с данными 

Давайте подгрузим все данные, которые у нас есть и подумаем как лучше с ними работать так, чтобы не запутаться и не сойти с ума. Все данные весят около 2 гигов. На самом деле я рекомендовал бы работать только с актуальными реально необходимыми для текущего куска задачи данными. 

In [1]:
import pickle

# Подгружаем все данные, что у нас есть 
with open('data/ria_news_2018.pickle', 'rb') as f:
    ria_news = pickle.load(f)
    
with open('data/ria_vk_posts', 'rb') as f:
    vk_news = pickle.load(f)
    
with open('data/ria_vk_comments', 'rb') as f:
    vk_comments = pickle.load(f)
    
print('Всего новостей за 2018 год:', len(ria_news))
print('Всего новостей в группе вк', len(vk_news))
print('Всего комментариев в группе вк', len(vk_comments))

Всего новостей за 2018 год: 201797
Всего новостей в группе вк 32277
Всего комментариев в группе вк 4040551


Каждая новость, пост, комментарий - это небольшой питоновский словарик. Посмотрим на примеры.

In [2]:
# Иньересующие нас поля: 
# category - рубрика с сайта
# date - дата публикации 
# keywords - ключевые слова
# shows - число просмотров новости 
# text - текст новости 
# title - заголовок новости
# href - ссылка на новость

ria_news[0]

{'category': 'В мире',
 'date': '2018-12-31',
 'day': '31',
 'href': '/20181231/1548961410.html',
 'image': 'https://cdn25.img.ria.ru/images/153303/06/1533030626_0:247:3501:2216_1036x0_80_0_0_291c92f1d31f7b2a613588e6a96164cb.jpg',
 'keywords': 'биньямин нетаньяху, израиль, в мире',
 'month': '12',
 'shows': '728',
 'snippet': 'Премьер-министр Израиля Биньямин Нетаньяху не намерен уходить в отставку в случае предъявления ему обвинений в коррупции, сообщает израильский телеканал i24... РИА Новости, 31.12.2018',
 'tags': 'Биньямин Нетаньяху, Израиль, В мире',
 'text': 'МОСКВА, 31 дек - РИА Новости. Премьер-министр Израиля Биньямин Нетаньяху не намерен уходить в отставку в случае предъявления ему обвинений в коррупции, сообщает израильский телеканал i24 News. В начале декабря полиция рекомендовала генпрокуратуре предъявить обвинения в коррупции Нетаньяху и его супруге Саре по итогам расследования дела об их отношениях с руководством крупнейшей в стране телекоммуникационной компании "Безек"

In [3]:
# Интересующие нас поля: 
# date - дата поста (число секунд прошедшее с 1 января 1970 года)
# id - идентификатор поста 
# text - содержание поста
# title - заголовок новости 
# url - ссылка на новость
# likes - число лайков 
# comments - число комментов под постом

vk_news[10]

{'likes': 144,
 'reposts': 8,
 'comments': 151,
 'text': '',
 'date': 1549047725,
 'from_id': -15755094,
 'id': 24005074,
 'owner_id': -15755094,
 'url': 'https://ria.ru/20190201/1550257362.html',
 'title': 'Лукашенко поручил убрать из школьных программ "все лишнее"'}

In [4]:
# Интересующие нас поля: 
# date - дата поста (число секунд прошедшее с 1 января 1970 года)
# id - id комментария 
# post_id - id поста, к которому был оставлен комментарий 
# likes - число лайков на комментарии 
# text - текст комментария 

vk_comments[0]

{'id': 24003685,
 'from_id': 529161030,
 'date': 1549038419,
 'text': 'встану утром рано, выпью чашку ртути и пойду подохну в этом институте',
 'likes': 39,
 'post_id': 24003681,
 'group_id': '15755094'}

Познакомились? Теперь попробуйте побороть страх пустоты. Код придётся писать чуть ли не с нуля. 

## Задание 1

__[5 баллов]__ Возьмите данные по твиттеру (они есть в общей папке) и обучите классификатор негативных комментариев. Попробуйте обучить две разные модели. Почему вы решили предобрабатывать данные для моделирования именно так, как решили? Выберите порог для отсечки негатива. 

__[2 баллов]__ Прогоните получившуюся модель по корпусу комментариев из вконтакте. Посмотрите на топ-10 самых негативных комментариев. 

__[2 балла]__ Посчитайте для каждой новости (`post_id`) среднее количество негативных комментариев. Какие 5 новостей пользователи восприняли негативнее всего (среди тех, где есть хотябы 5 комментариев)?

In [ ]:
# Ваш код почти с чистого листа :) 

__[1]__ Попробуем построить индекс негатива. Вытащите все новости, в текстах которых фигурирует фамилия Путин. Постройте дневной индекс негатива к Путину в новостях за весь рассматриваемый период. Не забудьте перевести timestamp (`date`) в нормальные даты пакетом datetime. При строительстве индекса сгладте его скользящим средним с окном $7$ (метод rolling_mean в pandas). 

In [ ]:
# Ваш код

----------------------------------------------

## Задание 2

Возьмите корпус новостей с сайта РИА-новости. Если он не помещается в памяти, возьмите корпус новостей из вконтакте. Он более компактный. 

__[5]__ обучите word2vec на этом корпусе. 

__[1]__ Решите примеры:
     * москва + екатеринбург - собянин
     * спартак - москва + санкт-петербург
     * иркутск - байкал + сочи
     * чай - лимон + кофе
     * авиакомпания - аэрофлот + ржд

__[1]__ Найдите лишнее, попробуйте интерпретировать результаты
     * магазин, супермаркет, рынок, тц
     * теннис, хоккей, футбол, дзюдо
     * кошка, собака, попугай, кролик
     * коми, дагестан, башкирия, камчатка
     
__[1]__ Найдите топ-10 слов с самым большим значением первой координаты. Можно ли как-то проинтерпретировать эту координату? Сделайте то же самое ещё с четыремя рандомными координатами. Попробуйте понять, за что они отвечают в нашем векторном пространстве. 

In [ ]:
# Ваш код

__[2]__ скачайте с сайта [Rusvectors](https://rusvectores.org/ru/models/) датасеты для оценки качества w2v моделей (смотрите раздел оценка качества моделей). Оцените качество своей модели. (для датасета `ru_analogy` можно использовать встроенный в модель метод accuracy.) 

In [ ]:
# Ваш код

__[Без баллов. Если станет скучно и есть желание порефлексировать (возможно, в этом нет смысла).]__ 

* На какое число надо умножить вектор 'хороший', чтобы самым близким словом к итоговому вектору оказалось слово 'лучший'? 
* На какое число надо умножить слово 'плохой', чтобы самым близким словом к итоговому вектору оказалось слово 'худший'? 

Какой из коэффициентов больше? Кем стать легче, лучшим или худшим (из плохого или хорошего)? 

* Какое слово обладает самой маленькой нормой и ближе всех находится к нулевому вектору? Есть ли в этом хоть какой-то смысл? 

In [ ]:
# Ваш код

----------------------------

## Задание 3 

__[5]__ Обучите на корпусе новостей тематическую модель c 20 темами. Проинтерпретируйте каждую тему, если это возможно. 

__[3]__ Вытащите из выборки все новости, соответствующие словам Путин, Навальный, Обама, Трамп. Постройте для этих новостей тематические профили. Насколько сильно они отличаются между собой? 

__[1]__ Найдите в новостном копусе с помощью `pymorthy2` все фамилии (как смотрите ниже). Постройте для каждой высокочстотной фамилии тематический профиль (порог выберите сами на своё усмотрение). Перегоните все профили с помощью TSNE в двумерное пространство. Постройте визуализацию. Получилось ли так, что какие-то фамилии выделились в отдельные кластеры?  Грамемы пакет берёт из [словаря Opencorpora](http://opencorpora.org/dict.php?act=gram) __Важно:__ можно при желании использовать `pymystem`.

__[1]__ Проделайте TSNE-преобразования со всеми word2vec представлениями фамилий. Дальше снова сделайте TSNE с тем же значением перплексии, что и в предыдущем пункте, и постройте картинку. Стали ли кластеры более явными? 

In [9]:
import pymorphy2
morph.parse('обама')

[Parse(word='обама', tag=OpencorporaTag('NOUN,anim,masc,Sgtm,Surn sing,nomn'), normal_form='обама', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'обама', 3106, 0),))]

In [24]:
# Если среди тэгов есть 'Surn', pymorthy думает, что мы работаем с фамилиями. 

words = ['путин', 'обама', 'огурец', 'еда', 'школа', 'козлов', 'потапова']
prob_thresh = 0.3 # выберите сами порог по своим соображениям

for word in words:
    for p in morph.parse(word):
        if 'Surn' in p.tag and p.score >= prob_thresh:
            print(word)

путин
обама
козлов
потапова
потапова
потапова


In [ ]:
# Ваш код

Надеюсь, что вы справились и было интересно!